In [55]:
import os
import numpy as np
import pandas as pd
from IPython.display import display

dataSource = 'participant' #'pilot'

path = './data/interaction_ros_logs'

df = pd.read_csv(os.path.join(path,'{}_durations.csv'.format(dataSource)))
display(df)

,PID,Trial,Duration-1,Duration-2,Duration-3,Duration-4,Duration-5,Duration-6,Duration-7,Duration-8,Duration-9,Duration-10,Duration-11,Duration-12
0,45,1,3.5200,4.53,32.61,6.03,27.60,5.62,34.61,4.29,24.09,7.57,22.10,14.54
1,45,2,3.5144,6.59,32.64,4.85,27.61,6.60,34.60,5.85,24.10,5.88,22.10,9.06
2,45,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,46,1,3.5200,14.97,32.61,7.94,27.62,7.79,34.60,10.42,24.09,7.36,22.09,5.31
4,46,2,3.5200,12.83,32.59,10.25,27.61,9.44,34.61,8.15,24.10,8.19,22.09,6.47
5,46,3,3.5100,2.55,32.60,6.07,27.60,6.07,34.60,11.31,24.09,20.33,22.10,NaN
6,47,1,3.5274,1.05,32.62,5.21,27.61,4.72,34.60,6.06,24.09,5.64,22.10,6.11
7,47,2,3.5200,8.91,32.64,4.38,27.60,2.49,34.60,4.61,24.10,3.49,22.09,4.01
8,48,1,3.5200,11.45,32.11,2.89,29.10,1.52,36.10,0.71,24.09,2.10,22.09,1.93
9,48,2,3.5100,5.33,32.09,1.09,29.10,2.73,36.11,0.03,24.09,0.68,22.12,18.66


In [56]:
df.keys()

Index(['PID', 'Trial', 'Duration-1', 'Duration-2', 'Duration-3', 'Duration-4',
       'Duration-5', 'Duration-6', 'Duration-7', 'Duration-8', 'Duration-9',
       'Duration-10', 'Duration-11', 'Duration-12'],
      dtype='object')

In [57]:
durations = df.loc[:, ['Duration-{}'.format(i) for i in range(1,13)]].to_numpy()
# evens [0,2,4,...] (duration-1, duration-3, duration-5) neglect time
# odds [1,3,5,...] (duration-2, duration-4, duration-6) interaction time

neglect = durations[:,0:13:2]
interact = durations[:, 1:13:2]

rad_table = np.zeros((durations.shape[0],int(durations.shape[1]/2)+1))

# col 0 is total rad
totalNeglect = np.nan_to_num(np.sum(neglect,axis=1))
totalInteract = np.nan_to_num(np.sum(interact,axis=1))
rad_table[:,0] = np.true_divide(totalInteract, np.add(totalNeglect, totalInteract))

# col 1-6 are step rad
for i in range(0,6):
    rad_table[:,i+1] = np.true_divide(interact[:,i], np.add(interact[:,i], neglect[:,i]))
    
rdf = pd.DataFrame(data=rad_table, columns=['Total']+['Step-{}'.format(i+1) for i in range(0,6)])
idf = df.loc[:, ['PID','Trial']]

result = pd.concat([idf,rdf], axis=1)
display(result)

C:\Users\curth\Anaconda3\envs\R-enabled\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


,PID,Trial,Total,Step-1,Step-2,Step-3,Step-4,Step-5,Step-6
0,45,1,0.227567,0.562733,0.156056,0.169175,0.110283,0.239103,0.396834
1,45,2,0.211729,0.652191,0.129368,0.192926,0.144623,0.196131,0.290757
2,45,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,46,1,0.271228,0.809627,0.195808,0.219994,0.231453,0.234022,0.193796
4,46,2,0.276858,0.784709,0.239262,0.254791,0.190599,0.253639,0.226541
5,46,3,0.000000,0.420792,0.156969,0.180279,0.246352,0.457677,NaN
6,47,1,0.166092,0.229388,0.137721,0.145994,0.149041,0.189707,0.216590
7,47,2,0.161737,0.716814,0.118314,0.082752,0.117572,0.126495,0.153640
8,48,1,0.122904,0.764863,0.082571,0.049641,0.019288,0.080183,0.080350
9,48,2,0.162470,0.602941,0.032851,0.085768,0.000830,0.027453,0.457577


In [58]:
result.to_csv(os.path.join(path, '{}_rad_table.csv'.format(dataSource)), index=False, header=True)